### Import Packages and Set Constants

In [ ]:
import numpy as np
import opensimplex as osi
import pyvista as pv
import pyvistaqt as pvqt
import cmocean as cmo
from numpy.typing import NDArray
from scipy.spatial import KDTree

osi.random_seed()

np.set_printoptions(suppress=True)

init_roughness = 1.5
init_strength = 0.4
roughness = 2.5
persistence = 0.5
radius = 6378100
recursion = 6
octaves = 8
zmin = -10000
zmax = 10000
zrange = zmax-zmin
zscale = 0.5e-5
num_plates = 48

### Initialize Mesh & Create Elevation Noise Feeder Arrays

In [ ]:
mesh = pv.Icosphere(radius=radius, nsub=recursion, center=(0.0,0.0,0.0))
raw_elevations: NDArray[np.float64] = np.zeros(len(mesh.points), dtype=np.float64)
roughness_values = np.array(object=[(init_roughness * (roughness**i)) / radius for i in range(octaves)])
strength_values = np.array(object=[(init_strength * (persistence**i)) / radius for i in range(octaves)])

### Generate Elevations

In [ ]:
for i in range(octaves):
    rough_verts = mesh.points * roughness_values[i]
    octave_elevations = np.ones(len(mesh.points), dtype=np.float64)

    for v in range(len(rough_verts)):
        octave_elevations[v] = osi.noise4(
            x=rough_verts[v][0],
            y=rough_verts[v][1],
            z=rough_verts[v][2],
            w=1,
        )

    raw_elevations += octave_elevations * strength_values[i] * radius

### Create Scalars and Elevations Data Array

In [ ]:
# Calculate elevation scalars.
elevation_scalars = (raw_elevations + radius) / radius

mesh.point_data["Elevation Scalars"] = elevation_scalars

mesh.points[:, 0] *= elevation_scalars  # type: ignore
mesh.points[:, 1] *= elevation_scalars  # type: ignore
mesh.points[:, 2] *= elevation_scalars  # type: ignore

emin = np.min(elevation_scalars)
emax = np.max(elevation_scalars)
erange = emax - emin

# Rescale elevations to zrange.
rescaled_elevations = ((elevation_scalars - emin) / erange) * zrange + zmin
mesh.point_data["Elevations"] = rescaled_elevations

# Create landform array.
landforms = rescaled_elevations >= 0 # Each point where rescaled elevation >= 0 is considered land.
mesh.point_data["Landforms"] = landforms

# Output
print("Raw Elevations Range:")
print(f"Min: {np.min(raw_elevations)}   Max: {np.max(raw_elevations)}\r\n")

print("Elevation Scalars:")
print(f"Min: {emin}   Max: {emax}\r\n")

print("Rescaled Elevations:")
print(f"Min: {np.min(rescaled_elevations)}   Max: {np.max(rescaled_elevations)}\r\n")

print("Landforms:")
print(landforms)

### Warp Mesh to Show Relief

In [ ]:
mesh.compute_normals(inplace=True)
mesh.warp_by_scalar(scalars="Elevations", factor=zscale, inplace=True)

### Create Tectonic Plates

In [ ]:
plate_centers = mesh.points[np.random.choice(len(mesh.points), num_plates, replace=False)]

# Ensure plate_centers contains only finite values
if not np.all(np.isfinite(plate_centers)):
    raise ValueError("plate_centers contains NaN or infinite values")

tree = KDTree(data=plate_centers)
distances, plate_indices = tree.query(x=mesh.points)
plate_sorted_indices = np.argsort(a=plate_indices)
plate_landmask = np.column_stack((plate_indices, landforms))

mesh.point_data["Tectonic Plates Mask"] = plate_landmask

mesh.point_data["Tectonic Plates"] = plate_indices

### Move the Plates

In [ ]:
# Generate random vectors in the XY plane for each plate
random_vectors = np.random.rand(num_plates, 2) * 2 - 1  # Random values in range [-1, 1]
normalized_vectors = random_vectors
normalized_vectors /= np.linalg.norm(random_vectors, axis=1, keepdims=True)  # Normalize vectors

# Apply the random vectors to the mesh points based on plate indices
for plate_index in range(num_plates):
    print(plate_index)
    print()
    mask = plate_indices == plate_index
    mesh.points[plate_index, 0] += normalized_vectors[plate_index, 0]  # Apply X component
    print(mesh.points[mask, 0])
    print()
    mesh.points[plate_index, 1] += normalized_vectors[plate_index, 1]  # Apply Y component
    print(mesh.points[mask, 1])
    print()

### Plot

In [ ]:
pv.set_plot_theme("dark")  # type: ignore
pv.global_theme.lighting = False

plotter = pv.Plotter(notebook=True)

tectonic_plates_annotations: dict = {i: str(object=f"Plate {i}") for i in range(0, np.max(plate_indices))}

tectonic_color_map = pv.LookupTable(
    cmap="Accent",
    n_values=num_plates,
    flip=False,
    values=None,
    value_range=None,
    hue_range=None,
    alpha_range=None,
    scalar_range=(np.min(plate_indices), np.max(plate_indices)),
    log_scale=None,
    nan_color=None,
    above_range_color=None,
    below_range_color=None,
    ramp=None,
    annotations=tectonic_plates_annotations,
)

# plotter.add_mesh(
#     mesh,
#     scalars="Tectonic Plates",
#     cmap="Accent",
#     opacity=1,
#     # categories=True,
#     # annotations=tectonic_plates_annotations,
# )

plotter.add_mesh(
    mesh,
    scalars="Elevations",
    cmap="cmo.topo",
    categories=False,
    pickable=False,
    preference="point",
    style="surface",
    pbr=True,
    roughness=0.75,
    copy_mesh=False
)

# plotter.enable_depth_of_field()
# plotter.enable_anti_aliasing('ssaa')
plotter.show()